In [ ]:
import dalex as dx
import pandas as pd
import pickle
import xgboost as xgb
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
input_df = pd.read_csv('data/Bartek/new_preprocessed_dataset.csv')
y = input_df.loc[:,'Attrition']
X = input_df.drop('Attrition', axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14)

In [ ]:
path_lr = 'modele/l1_log_reg.p'
lr = pickle.load(open( path_lr, "rb" ))
lr_explainer = dx.Explainer(lr, X_train, y_train, label='Logistic Regresion')

## Obserwacja błędnie zaklasyfikowana przez oba modele

In [ ]:
lr_errors = np.round(lr_explainer.predict(X_test)) != y_test
incorrect = X_test[lr_errors]

In [ ]:
incorrect.index

In [ ]:
index = 8425

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('LR prediction : ' + str(lr_explainer.predict(observation)[0]))


Klient zrezygnował z usługi, lecz model LR przewidział jego rezygnację z prawdopodobieństwem 31%

### Break Down

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index])
lr_pp.plot(max_vars=24)

Zmienna `Total_Revolving_Bal` = 0 zwiększa prawdopodobieństwo rezygnacji o 9%, `Total_Ct_Chng_Q4_Q1` = 0.5 zwiększa o 10%. Zmienne `Total_Relationship_Count` = 5 oraz `Contacts_Count_12_mon` = 2 przyczyniają się do zmiejszenia predykcji rezygnacji. 

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index],type='shap')
lr_pp.plot(max_vars=24)

Dla Shapley value zmienne mają podobny udział w predykcji jak dla Break Down, jednak tyma razem większy wpływ ma zmienna `Total_Revolving_Bal`, jest na drugim miejscu `Total_Ct_Chng_Q4_Q1`.

In [ ]:
lr_lime = lr_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
lr_lime.show_in_notebook()


Dla metody LIME także duży wpływ na predykję mają `Total_Revolving_Bal`, `Total_Ct_Chng_Q4_Q1`, `Total_Relationship_Count`, `Contacts_Count_12_mon`. Dodatkowo LIME wykrył wpływ zmiennej `Card_Category`, która w poprzednich metodach znoajdowała się dosyć nisko, oraz ocenił, że jej wartość równa 1 zmiejsza prawdopodobieństwo odejścia z banku.

### Ceteris Paribus

In [ ]:
lr_cp = lr_explainer.predict_profile(X_test.loc[index])

lr_cp.plot(facet_ncol=4)

Największę zmieny w predykcji są dla zmiennych  `Total_Ct_Chng_Q4_Q1`, `Total_Relationship_Count`, `Contacts_Count_12_mon` na które już zwróciliśmy uwagę w poprzednich metodach. Zaóważmy też, że zmiany dla zmiennej`Card_Category`, którą zaóważyliśmy w metodzie LIME, może zmienić predykcję prawdopodobieństwa rezygnacji z 30% do prawię 60%. Dodatkowo możemy, zaóważyć duże zmiiany w predykcji dla zmiennej `Months_Inactive_12_mon` która może zmienić predykcję od 12% do prawię 60%.

### Podsumowanie

- duży wpływ do prawidłowej predykcji miały zmienne `Total_Revolving_Bal`, `Total_Ct_Chng_Q4_Q1`
- największy wpływ na predykję wykazywały zmienne `Total_Revolving_Bal`, `Total_Ct_Chng_Q4_Q1`, `Total_Relationship_Count`, `Contacts_Count_12_mon`, `Card_Category` oraz `Months_Inactive_12_mon`

## Obserwacja bardzo dobrze klasyfikowana

In [ ]:
treshhold = 0.00005

lr_errors = np.abs(lr_explainer.predict(X_test) - y_test)
super_correct = X_test[lr_errors < treshhold]


In [ ]:
super_correct.index

Mniej bardzo dobrze sklasyfikowanych obserwacji niż dla modelii RF i XGB.

In [ ]:
index = 805

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('LR prediction : ' + str(lr_explainer.predict(observation)[0]))

Model Regresji Logicznej przewidział, że klijent zrezygnuje z prawdopodobieństwem równym prawię 0%.


### Break Down

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index])
lr_pp.plot(max_vars=24)

Według metody Break Down model dokonał predykcji na podstawię zmiennej `Total_Ct_Chng_Q4_Q1` = 2.5. W badaniu błędnie sklasyfikowanej obserwacji ta zmienna także miała duży wpływ do predykcji, lecznie była to jedyna zmienna której udział był istotny.


### Shapley Values

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index],type='shap')
lr_pp.plot(max_vars=24)

Podobnie jak w poprzedniej metodzie, największy wplyw do predykcji miała zmienna `Total_Ct_Chng_Q4_Q1`. Dodatkowo metoda Shapley Values pokazała wpływ `Months_Inactive_12_mon`, `Total_Trans_Amt`,`Contacts_Count_12_mon`, `Total_Revolving_Bal`  które miały ponad 3 razy mniejszy wplyw od `Total_Ct_Chng_Q4_Q1`.

In [ ]:
lr_lime = lr_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
lr_lime.show_in_notebook()

Podobnie jak w analizie poprzedniej obserwacji, oprócz wcześniej wyróznionych zmiennych metoda LIME wyróżnia dodatkowo w wpływ zmiennej `Card_Category`.

### Ceteris Paribus

In [ ]:
lr_cp = lr_explainer.predict_profile(X_test.loc[index])

lr_cp.plot(facet_ncol=4)

Metoda Ceteris Paribus pododnie jak Break Down wyróżnia tylko zmienną `Total_Ct_Chng_Q4_Q1`. Zmienna ta to iloraz sumy transakcji z 4. kwartału oraz 1. kwartału i pokazuje czy liczba transakcji klienta się zwiększa czy zmniejsza. Zmienna `Total_Ct_Chng_Q4_Q1` równa 2.5 co może oznaczać, że klient wykonuje 2.5 razy wiecej transakcji niż na początku 12 miesięcy. Dla modelu Regresji Logicznej jest to bardzo dobry wyznacznik prawdopodobnie dlatego, że aktywny klient pewnie nie będzie chciał zrezygnować. Zaóważmy, że predykcja prawdopodobieństwa rezygnacji zacznie rosną dopiero gdy `Total_Ct_Chng_Q4_Q1` zacznie spadać poniżej 1 czyli aktywności klienta zmaleje.Jednak nawet jeżeli osiągnie wartość 0 to predykcja będzie wynościć 26% ponieważ posotałe zmienne na które zwraca uwagę Shapley Values mają także wpływają na spadek predykcji.


### Podsumowanie
- dla obserwacji o bardzo dobrej predykcji najważniejszą zmienna jest `Total_Ct_Chng_Q4_Q1`
- zmienna ta według modelu jest wyznacznikiem aktywności klienta jeżeli osiąga watości 1 lub więcej

## Obserwacja niepewnie zaklasyfikowana przez oba modele

In [ ]:
lr_ff_a = lr_explainer.predict(X_test) > 0.42 
lr_ff_b = lr_explainer.predict(X_test) < 0.57
ff = X_test[lr_ff_a & lr_ff_b]

In [ ]:
ff.index

Więcej niż dla modelii RF i XGB

In [ ]:
index = 8546

In [ ]:
print('Label: ' + str(y_test.loc[index]))
observation = X_test.loc[index].to_frame().transpose()
print('LR prediction : ' + str(lr_explainer.predict(observation)[0]))

Wybrany klient nie zrezygnował z usługi, lecz model z małą pewnościa 54% przewidział rezygnację.

### Break Down

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index])
lr_pp.plot(max_vars=24)

Dla metody Break Down największy wpływ na predykcję miały zmienne `Total_Revolving_bal` = 0 oraz `Total_relationship_Count` = 2. `Total_Revolving_bal` (suma wykorzystanego debetu z 12 miesięcy) oznacza że klient nie pozostawia debetu pod koniec miesiąca lub że nie korzysta z debetu co może oznaczać, że nie jeste zainteresowany oderowaną usułgą.


### Shapley Values

In [ ]:
lr_pp = lr_explainer.predict_parts(X_test.loc[index],type='shap')
lr_pp.plot(max_vars=24)

Za pomocą metody Dreak Down możemy zaóważyć duży wpływ zmiennej `Contacts_Count_12_mon` = 3 która oznacza znaczą liczbę kontaktów z bankiem z ostatnich 12 miesięcy. Możliwe że może to oznaczać skargi klieta. Dodatkowo `Months_Inactive_12_mon` = 3 także wpływa na wzrost predykji. Jest to dosyć zrozumiałe, że brak aktywnosći klienta może oznaczać, że będzie chciał on zrezygnować z usługi.

### LIME

In [ ]:
lr_lime = lr_explainer.predict_surrogate(X_test.loc[index], type='lime', seed=10)
lr_lime.show_in_notebook()

Oprócz wczęsniej wymienionych zmiennych, tak jak w pozostałych obserwacjach, metoda LIME zwraca uwagę na zmienną `Card_Category` równą 1.


### Ceteris Paribus

In [ ]:
lr_cp = lr_explainer.predict_profile(X_test.loc[index])

lr_cp.plot(facet_ncol=4)

Możemy zaóważyć duze zmiany w predykcji dla wczesniej wspomnianych zmiennych`Total_Revolving_bal`, `Total_relationship_Count`, `Contacts_Count_12_mon`, `Months_Inactive_12_mon`. Duży wpływ mają także zmiany zmiennych `Total_Trans_Amt` ,`Total_Relationship_Count` oraz `Total_Ct_Chng_Q4_Q1`. Dla `Total_Ct_Chng_Q4_Q1`,  `Months_Inactive_12_mon` oraz `Contacts_Count_12_mon` obserwacji przyjmuje wartości "średnie" i wystarczy by był zmiejsze lub większe by predykcja nie była niepewna. 


### Podsumowanie
- obserwacja posiadała wiele średnich wartości dla zmiennych, które mogły by pewniej ją zaklasyfikować 
- aktywność użytkowanika mogłaby sugerować jego brak zainteresowania usługami lub niezadowolenie z nich, chociaż jest on klientem który nie zrezygnował 
- zmiany wartości zmiennych `Total_Ct_Chng_Q4_Q1`,  `Months_Inactive_12_mon` oraz `Contacts_Count_12_mon` zmniejszyły by niepewność predykcji.